In [ ]:
# Set the work directory to the imaginaire root.
import os, sys, time
import pathlib

root_dir = pathlib.Path().absolute().parents[0]
os.chdir(root_dir)
print(f"Root Directory Path: {root_dir}")

In [2]:
# Import Python libraries.
import numpy as np
import torch
import k3d
import json
import trimesh
import plotly.graph_objs as go
from collections import OrderedDict
# Import imaginaire modules.
from submodules.colmap.scripts.python.read_write_model import read_model
# from tools import camera, visualize
from tools.camera import quaternion
from tools.visualize import k3d_visualize_pose, plotly_visualize_pose
from process_data.convert_tnt_to_json import load_transformation, align_gt_with_cam
from tools.camera_utils import cubic_camera, grid_camera, around_camera, up_camera, bb_camera
from tools.math_utils import inv_normalize_pts

In [ ]:
# Read the COLMAP data.
# colmap_path = "datasets/lego_ds2"
scene = 'Barn'
colmap_path = f"/your/path/tnt/{scene}"
# read piont clouds from lidar # point cloud
pcd = trimesh.load(os.path.join(colmap_path, '{}.ply'.format(colmap_path.split('/')[-1])))
# scene = 'c49a8c6cff'
# colmap_path = f"/your/path/ScanNet++/{scene}/dslr"
# pcd = trimesh.load(os.path.join(colmap_path, '../scans/mesh_aligned_0.05.ply'))
view_sample_camera = False
cameras, images, points_3D = read_model(path=f"{colmap_path}/sparse", ext=".bin") # w2c extrinsics
# Convert camera poses.
images = OrderedDict(sorted(images.items()))
qvecs = torch.from_numpy(np.stack([image.qvec for image in images.values()]))
tvecs = torch.from_numpy(np.stack([image.tvec for image in images.values()]))
# Rs = camera.quaternion.q_to_R(qvecs)
Rs = quaternion.q_to_R(qvecs)
poses = torch.cat([Rs, tvecs[..., None]], dim=-1)  # [N,3,4]  w2c
print(f"# images: {len(poses)}")
print("camera height: {}".format(poses[:, 1, 3].mean()))

# # Get the sparse 3D points and the colors. colmap
# xyzs = torch.from_numpy(np.stack([point.xyz for point in points_3D.values()]))
# rgbs = np.stack([point.rgb for point in points_3D.values()])
# rgbs_int32 = (rgbs[:, 0] * 2**16 + rgbs[:, 1] * 2**8 + rgbs[:, 2]).astype(np.uint32)
# print(f"# points: {len(xyzs)}")


if os.path.exists(os.path.join(colmap_path, f'{scene}_trans.txt')):
    trans = load_transformation(os.path.join(colmap_path, f'{scene}_trans.txt'))
    pcd.vertices = align_gt_with_cam(pcd.vertices, trans)
    
xyzs = pcd.vertices[::500]
# xyzs = pcd.vertices
rgbs = np.random.randint(0, 255, xyzs.shape)
rgbs_int32 = (rgbs[:, 0] * 2**16 + rgbs[:, 1] * 2**8 + rgbs[:, 2]).astype(np.uint32)
print(f"# points: {len(xyzs)}")

In [4]:
vis_depth = 0.2

In [5]:
# Visualize the bounding sphere.
json_fname = f"{colmap_path}/meta.json"
with open(json_fname) as file:
    meta = json.load(file)
trans = np.array(meta["trans"])
scale = np.array(meta["scale"])
# ------------------------------------------------------------------------------------
# These variables can be adjusted to make the bounding sphere fit the region of interest.
# The adjusted values can then be set in the config as data.readjust.center and data.readjust.scale
readjust_center = np.array([0., 0., 0.])
readjust_scale = np.array([1., 1., 1.]) # * 1.1
# save adjusted values
readjust = {
    'scale': readjust_scale.tolist(),
    'trans': readjust_center.tolist()
}
redjust_fname = f'{colmap_path}/readjust.json'
with open(redjust_fname, "w") as outputfile:
    json.dump(readjust, outputfile, indent=2)
# ------------------------------------------------------------------------------------
if trans.ndim == 1:
    trans += readjust_center
scale *= readjust_scale
# Make some points to hallucinate a bounding sphere.
# sphere_points = np.random.randn(100000, 3)
sphere_points = np.random.rand(100000, 3) * 2 - 1
# sphere_points = sphere_points / np.linalg.norm(sphere_points, axis=-1, keepdims=True) # Unit sphere
# sphere_points[:, 0] = -1 # up
for i in range(3): sphere_points[i::3, i] = sphere_points[i::3, i] / np.abs(sphere_points[i::3, i]) # Unit cube
sphere_points = np.concatenate([sphere_points, np.zeros([1, 3])], axis=0) # center point
# sphere_points[-1, 0] = 5

sphere_points = inv_normalize_pts(sphere_points, trans, scale)

# sphere_points[:, 1] = -1.1

# sample up cameras
if view_sample_camera:
    height = poses[:, 1, 3].mean()
    # height = -1
    # sample_poses = cubic_camera(200, trans, scale)
    # sample_poses = around_camera(500, trans, scale, height)
    # sample_poses = bb_camera(500, trans, scale, height, up=False, around=True)
    sample_poses = bb_camera(200, trans, scale, height=height, up=True, around=True, bidirect=True) # , look_mode='direction'
    # sample_poses = up_camera(500, trans, scale)
    # sample_poses = grid_camera(trans, scale)

    # sample_poses = torch.from_numpy(poses[:, :3])
    sample_poses = sample_poses[:, :3]

    # poses = torch.cat([poses, sample_poses], dim=0)
    poses = sample_poses # [::6]
    # print(f"# poses: {len(poses)}")

    # print(f"center: {trans[:3, 3:].T}")
    # print(f"scale: {scale}")
    # print("up: {}".format(trans[1, 3] - scale[1] * 0.5))
    # print(f"max: {sphere_points.max(0)}")
    # print(f"min: {sphere_points.min(0)}")

In [ ]:
# You can choose to visualize with Plotly...
x, y, z = *xyzs.T,
colors = rgbs / 255.0
sphere_x, sphere_y, sphere_z = *sphere_points.T,
sphere_colors = ["#4488ff"] * len(sphere_points)
sphere_size = [0.5] * len(sphere_points)
sphere_colors[-1] = "#ff0000" # #ff4444 center point
# sphere_size[-1] = 5
# traces_poses = visualize.plotly_visualize_pose(poses, vis_depth=vis_depth, xyz_length=0.02, center_size=0.01, xyz_width=0.005, mesh_opacity=0.05)
traces_poses = plotly_visualize_pose(poses, vis_depth=vis_depth, xyz_length=0.02, center_size=0.01, xyz_width=0.005, mesh_opacity=0.05)
trace_points = go.Scatter3d(x=x, y=y, z=z, mode="markers", marker=dict(size=0.4, color=colors, opacity=0.7), hoverinfo="skip")
trace_sphere = go.Scatter3d(x=sphere_x, y=sphere_y, z=sphere_z, mode="markers", marker=dict(size=sphere_size, color=sphere_colors, opacity=0.7), hoverinfo="skip")
traces_all = traces_poses + [trace_points, trace_sphere]
layout = go.Layout(scene=dict(xaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              yaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              zaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              xaxis_title="X", yaxis_title="Y", zaxis_title="Z", dragmode="orbit",
                              aspectratio=dict(x=1, y=1, z=1), aspectmode="data"), height=800)
fig = go.Figure(data=traces_all, layout=layout)
fig.show()

In [ ]:
# ... or visualize with K3D.
plot = k3d.plot(name="poses", height=800, camera_rotate_speed=5.0, camera_zoom_speed=3.0, camera_pan_speed=1.0)
# k3d_objects = visualize.k3d_visualize_pose(poses, vis_depth=vis_depth, xyz_length=0.02, center_size=0.01, xyz_width=0.005, mesh_opacity=0.05)
k3d_objects = k3d_visualize_pose(poses, vis_depth=vis_depth, xyz_length=0.02, center_size=0.01, xyz_width=0.005, mesh_opacity=0.05)
for k3d_object in k3d_objects:
    plot += k3d_object
plot += k3d.points(xyzs, colors=rgbs_int32, point_size=0.02, shader="flat")
plot += k3d.points(sphere_points, color=0x4488ff, point_size=0.01, shader="flat")
plot.display()
plot.camera_fov = 30.0